In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# Load the minimum required library to run the functions
from Utils_functions import Simulator_noGPU, corr
from numpy.random import uniform
from numpy import array, int64, where, concatenate, linspace
from scipy.integrate import cumtrapz

In [3]:
dt = 0.01 # The time step size for the simulation. This is the amount of time that passes in the model for each step of the simulation
DeltaT = 1 # The sampling period. This is the amount of time that passes in the model between each sample that is recorded
TotalT = 100 # The total time for the simulation. This is the total amount of time that the simulation is intended to represent
n_sim = 10
t_corr = 10

In [4]:
time_steps_amount = int64(TotalT/dt) # Number of steps
sampled_point_amount = int64(TotalT/DeltaT) # Number of sampled points
sampling_delta_time_steps = int64(DeltaT/dt) # Number of steps between samples
t = linspace(0., TotalT,sampled_point_amount) # Time array
    

In [5]:
mu_x = uniform(1.5e4, 4e4, size=(n_sim,1))
mu_y = uniform(1e4, 140e4, size=(n_sim,1))
k_x = uniform(3e-3, 16e-3, size=(n_sim,1))
k_y = uniform(1.5e-2, 30e-2, size=(n_sim,1))
k_int = uniform(1e-3, 6e-3, size=(n_sim,1))
tau = uniform(2e-2, 20e-2, size=(n_sim,1))
eps = uniform(0.5, 6, size=(n_sim,1))
D_x = uniform(5.5, 15.5, size=(n_sim,1))
D_y = uniform(1, 530, size=(n_sim,1))
sigma = uniform(4, 260, size=(n_sim,1))

In [6]:
mu_x = uniform(0.1, 0.5, size=(n_sim,1))
mu_y = uniform(0.1, 0.5, size=(n_sim,1))
k_x = uniform(0.1, 0.5, size=(n_sim,1))
k_y = uniform(0.1, 0.5, size=(n_sim,1))
k_int = uniform(0.1, 0.5, size=(n_sim,1))
tau = uniform(0.1, 0.5, size=(n_sim,1))
eps = uniform(0.1, 0.5, size=(n_sim,1))
D_x = uniform(0.1, 0.5, size=(n_sim,1))
D_y = uniform(0.1, 0.5, size=(n_sim,1))
sigma = uniform(0.1, 0.5, size=(n_sim,1))

In [7]:
theta = [mu_x, mu_y, k_x, k_y, k_int, tau, eps, D_x, D_y]

In [8]:
x_trace, f_trace, state = Simulator_noGPU(dt, DeltaT, TotalT, n_sim, theta)

In [9]:
S_red.shape

NameError: name 'S_red' is not defined

In [ ]:
Cxx[idx_corr].shape

(9,)

In [ ]:
mu_x.shape

(10, 1)

In [11]:
C_xxs = []
C_fxs = []
S_red_all = []
inputs = []
u_x = uniform(0.1, 0.5)
kappa_x = uniform(0.1, 0.5)
Di_x = uniform(0.1, 0.5)
for i in range(n_sim):
    Cxx= corr(x_trace[i], x_trace[i], sampled_point_amount, dt=DeltaT)
    Cfx = corr(f_trace[i], x_trace[i], sampled_point_amount, dt=DeltaT)
    C_xxs.append(Cxx)
    C_fxs.append(Cfx)
    
    S1 = cumtrapz(Cxx, x=t, axis=-1, initial=0)
    S1 = cumtrapz(S1, x=t, axis=-1, initial=0)
    idx_corr = where((t>0)*(t<t_corr))[0]
    S_red = ((Cxx[0]-Cxx[idx_corr])+((mu_x[i]*k_x[i])**2)*S1[idx_corr])/(D_x[i]*t[idx_corr])
    S_red_all.append(S_red)
    input = concatenate((Cxx[idx_corr],S_red))  # Use S_red[idx_corr] instead of S_red
    inputs.append(input)

print(f"The all x autocorrellation is {C_xxs[0]}")
print(f"The all f autocorrellation is {C_fxs[0]}")
print(f"The reduced energy production is {S_red_all[0]}")
print(f"The input of the infer is {inputs[0]}")

The all x autocorrellation is [ 1.83329925  1.71645045  1.61080534  1.52660314  1.43383845  1.32919318
  1.24158912  1.17669584  1.11394046  1.03596904  0.96708645  0.90125374
  0.84996815  0.78589376  0.72017801  0.6530115   0.58961168  0.51588738
  0.42180929  0.33631651  0.28434197  0.23788743  0.19403076  0.16095703
  0.14878822  0.1265107   0.09252358  0.06876363  0.05046893  0.01190399
 -0.02172212 -0.04647941 -0.08417471 -0.11028104 -0.11934423 -0.11478328
 -0.10705835 -0.07877115 -0.05328103 -0.04019319 -0.00755818  0.02661247
  0.03496916  0.04265173  0.03889183  0.03930025  0.03690783  0.03536901
  0.034084    0.03549239  0.04106604  0.02599017  0.0041629  -0.02135065
 -0.0434135  -0.06935994 -0.10601058 -0.13552196 -0.16410476 -0.18061482
 -0.22509909 -0.28020142 -0.32205282 -0.37059288 -0.42048451 -0.45894049
 -0.49381512 -0.51040655 -0.52615739 -0.54507744 -0.57250956 -0.59462618
 -0.6050028  -0.63895282 -0.66942392 -0.67616698 -0.6852091  -0.71393514
 -0.73067637 -0.73990

[array([1.71645045, 1.61080534, 1.52660314, 1.43383845, 1.32919318,
        1.24158912, 1.17669584, 1.11394046, 1.03596904, 0.67763669,
        0.6742599 , 0.65081747, 0.66328293, 0.69319344, 0.70452966,
        0.70035617, 0.70048989, 0.71396423]),
 array([0.23944068, 0.20453698, 0.17847004, 0.16262739, 0.15216811,
        0.13469162, 0.11929453, 0.1015399 , 0.09226634, 0.17363142,
        0.15967667, 0.1455879 , 0.1299396 , 0.11731792, 0.1141452 ,
        0.11127978, 0.11069443, 0.10691987]),
 array([0.36821204, 0.30229178, 0.25458724, 0.20781636, 0.18845314,
        0.16062963, 0.12003169, 0.08435754, 0.05762243, 0.26093794,
        0.26096051, 0.24284614, 0.23537638, 0.21264683, 0.20379651,
        0.2049555 , 0.20417087, 0.2003246 ]),
 array([1.50957194, 1.41465435, 1.36859323, 1.34059691, 1.32090827,
        1.32018448, 1.30652078, 1.25357003, 1.15483207, 0.5935808 ,
        0.60877923, 0.55927763, 0.5321594 , 0.5236072 , 0.51508258,
        0.52980902, 0.57575814, 0.64584998]),


In [12]:
Cxx[idx_corr].shape

(9,)

In [18]:
Cxx[idx_corr]

array([ 0.22398229,  0.16482741,  0.12118154,  0.08549176,  0.05088304,
        0.01692269, -0.01081265, -0.02399728, -0.01419307])

In [24]:
S_red[0].shape

(9,)

In [23]:
Cxx

array([ 2.76034733e-01,  2.23982294e-01,  1.64827406e-01,  1.21181539e-01,
        8.54917591e-02,  5.08830416e-02,  1.69226899e-02, -1.08126461e-02,
       -2.39972765e-02, -1.41930671e-02, -8.14127227e-04,  6.97788532e-04,
        8.17445968e-03,  5.21538533e-03, -9.12374399e-03, -1.53751686e-02,
       -1.22282824e-02, -1.88412959e-02, -3.58208620e-02, -4.84189441e-02,
       -6.25379409e-02, -7.39901029e-02, -7.04757717e-02, -7.30519480e-02,
       -6.19461294e-02, -6.63326302e-02, -7.08070233e-02, -7.09353432e-02,
       -5.45022758e-02, -3.48267365e-02, -3.15472038e-02, -2.53962731e-02,
       -1.43899950e-02,  6.24570461e-03,  2.56831094e-02,  2.25439943e-02,
        2.15740551e-02,  2.44452894e-02,  2.29018399e-02,  1.16199757e-02,
       -3.70977330e-03, -9.86195689e-03, -1.81070892e-02, -2.48677529e-02,
       -2.69955740e-02, -2.54237301e-02, -6.90945392e-03,  1.09803831e-02,
        2.37334586e-02,  3.26744839e-02,  4.18918184e-02,  4.23295801e-02,
        4.25124288e-02,  

In [11]:
C_xxs = []
C_fxs = []
S_red_all = []
inputs = []
for i in range(n_sim):
    Cxx= corr(x_trace[i], x_trace[i], sampled_point_amount, dt=DeltaT)
    Cfx = corr(f_trace[i], x_trace[i], sampled_point_amount, dt=DeltaT)
    C_xxs.append(Cxx)
    C_fxs.append(Cfx)
    
    S1 = cumtrapz(Cxx, x=t, axis=-1, initial=0)
    S1 = cumtrapz(S1, x=t, axis=-1, initial=0)

    idx_corr = where((t>0)*(t<t_corr))[0]

    S_red = ((Cxx[0]-Cxx[idx_corr])+((mu_x*k_x)**2)*S1[idx_corr])/(D_x*t[idx_corr])
    S_red_all.append(S_red)
    
    input = concatenate((Cxx[idx_corr],S_red))
    inputs.append(input)

print(f"The all x autocorrellation is {C_xxs[0]}")
print(f"The all f autocorrellation is {C_fxs[0]}")
print(f"The reduced energy production is {S_red_all[0]}")
print(f"The input of the infer is {inputs[0]}")

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
S2 = cumtrapz(Cfx - Cfx[0], x=t, axis=-1, initial=0)
    
S3 = cumtrapz(Cfx, x=t, axis=-1, initial=0)
S3 = -μ*k*cumtrapz(S3, x=t, axis=-1, initial=0)

t_corr = 10

idx_corr = where((t>0)*(t<t_corr))[0]

#S_red = 1+(S1[idx_corr]+S2[idx_corr]+S3[idx_corr])/(kBT*t[idx_corr])
S_red = 1-(S2[idx_corr]+S3[idx_corr])/(kBT*t[idx_corr]) # the energy production is to to the fluctuation-dissipation theorem
return concatenate((Cxx[idx_corr],S_red)) # return the correlation function and the reduced entropy production